In [ ]:
#Solve ToxicDetection through feature engineer into logistic regression
# Reference this tutorial for the logistic regression: 
##  https://www.kaggle.com/negation/pytorch-logistic-regression-tutorial
%load_ext autoreload
%autoreload 2


In [ ]:
#Input Data needs to be in array of size N x (2)
import utils 
import numpy as np
import pandas as pd
import os
import torch
from torch import nn
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')



In [ ]:
pd.set_option('display.max_colwidth', -1)
root_dir = os.path.abspath('.')
data_dir = os.path.join(root_dir, 'dataset')
train = pd.read_csv(os.path.join(data_dir,'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))


train_x, train_y = utils.featurize(train)
sampleIdx = np.random.choice(np.arange(len(train_y)), 15000, replace = False)
val_x, val_y= train_x[sampleIdx], train_y[sampleIdx]



In [ ]:


class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes=2, hidden_dim=1024):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):        
        #m = nn.Softmax()
        n = nn.functional.leaky_relu(self.linear(x))
        n2 = self.linear2(n)
        return n2
        

In [ ]:
#train on data_train with several epochs
epochs = 300 #Can also be changed. The higher this value, the more overfit. 
feature_dims = 21
batchSize = 32
classes = 6 #Binaray for toxic, or not
models = []
#model = model.cuda() #COMMENT OUT TO RUN ON GPU
#Todo: set loss criterion and optimizer
#Run loop of number of epoch, forward and backpass, loss then into optimizer

lossFns = []
optimizers = []

#Training
for c in range(classes):
    models.append(LogisticRegression(feature_dims))
    lossFns.append(nn.CrossEntropyLoss())
    optimizers.append(torch.optim.SGD(models[c].parameters(), lr = 1e-3))
    for epoch in range(epochs):
        sampleIdx = np.random.choice(np.arange(len(train_y)), batchSize, replace=False)
        x_s = torch.from_numpy(train_x[sampleIdx]).float().to(device)
        y_s = torch.from_numpy(train_y[sampleIdx]).to(device)
        #for i in range(batchSize): 
            #print(x_s[i].shape, y_s[i].shape)
        optimizers[c].zero_grad()
        scores = models[c](x_s)

        
        loss = lossFns[c](scores, y_s[:,c])
        loss.backward()
        optimizers[c].step()

        if (epoch) % 100 == 0:
                print ('Class: %d, Epoch: [%d/%d], Loss: %.4f' 
                       % (c, epoch, epochs,  loss.data[0]))



    


In [ ]:
correctCount = np.zeros((classes))
sampleIdx = np.random.choice(np.arange(len(train_y)), 300, replace = False)
val_x, val_y= train_x[sampleIdx], train_y[sampleIdx]
for c in range(classes):
    
    for i in range(len(val_x)):
        scores = models[c](torch.from_numpy(val_x[i]).float().to(device)).detach().numpy()
        if val_y[i,c]==np.argmax(scores, axis=0):
            correctCount[c] += 1
print(correctCount / len(val_x))


In [ ]:
#TODO:
#Optimize hyperparameters like batchsize, or learning rate or
#Try to add dropout, batch normalization to get highest val acc


In [ ]:
#TODO: 
#Now try test set, hopefully for accuracy as good as validation set